In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnext101_32x8d
from tqdm import tqdm
import time
import copy
import os
import csv
from torch.utils.data import Dataset
from PIL import Image

In [2]:
data_dir = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data'
batch_size = 8
num_workers = 4
num_classes = 100

# Data augmentations and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Datasets and Dataloaders
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}",
                                          transform=data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=num_workers)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            loop = tqdm(dataloaders[phase], total=len(dataloaders[phase]),
                        desc=f"{phase.capitalize()} Epoch {epoch+1}")

            for inputs, labels in loop:
                inputs = inputs.to(device)
                labels = labels.to(device)


                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                # print(labels.data)

                loop.set_postfix(loss=running_loss /
                                 ((loop.n + 1) * batch_size),
                                 acc=running_corrects.double().item() /
                                 ((loop.n + 1) * batch_size))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        scheduler.step()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m\
          {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model


In [4]:
# Initialize ResNext101
model = resnext101_32x8d(pretrained=True)

# Unfreeze all layers (no layers frozen)
for param in model.parameters():
    param.requires_grad = True

# Replace the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (finetune all parameters)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
num_epochs = 100
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                    num_epochs=num_epochs)

# Save the best model
torch.save(model.state_dict(), 'resnext101_alllayers.pth')
print('Model saved as resnext101_alllayers.pth')

Epoch 1/100
----------


Train Epoch 1: 100%|██████████| 2591/2591 [14:14<00:00,  3.03it/s, acc=0.517, loss=1.97] 


train Loss: 1.9678 Acc: 0.5170


Val Epoch 1: 100%|██████████| 38/38 [00:06<00:00,  5.91it/s, acc=0.579, loss=1.75]


val Loss: 1.7780 Acc: 0.5867
Epoch 2/100
----------


...

val Loss: 0.6368 Acc: 0.8433
Epoch 100/100
----------


Train Epoch 100: 100%|██████████| 2591/2591 [12:31<00:00,  3.45it/s, acc=0.914, loss=0.324]


train Loss: 0.3238 Acc: 0.9146


Val Epoch 100: 100%|██████████| 38/38 [00:03<00:00, 12.02it/s, acc=0.849, loss=0.636]


val Loss: 0.6445 Acc: 0.8600
Training complete in 1414m 36s
Best val Acc: 0.8733
Model saved as resnext101_alllayers.pth


In [6]:
# Load the trained model
model = resnext101_32x8d(num_classes=len(class_names))
model.load_state_dict(torch.load('resnext101_alllayers.pth'))
model = model.to(device)
model.eval()

# Define the transform (same as validation)
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

val_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/val/'
val_dataset = datasets.ImageFolder(val_folder, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,
                                         shuffle=False)

correct = 0
total = 0
all_predictions = []

with torch.no_grad():
    for i, (inputs, labels) in enumerate(tqdm(val_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        true_label = labels.item()
        predicted_label = preds.item()

        correct += (predicted_label == true_label)
        total += 1

        # Get the file path of the current sample
        img_path, _ = val_dataset.samples[i]
        all_predictions.append((img_path.split('.')[0], predicted_label))

# Save predictions to a CSV
fname = 'val_predictions_resnext101_alllayers.csv'
with open(fname, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(all_predictions)


accuracy = correct / total * 100
print(f'Validation Accuracy: {accuracy:.2f}%')

100%|██████████| 300/300 [00:08<00:00, 35.42it/s]

Validation Accuracy: 87.33%


In [7]:
# Load the Model
model = resnext101_32x8d(num_classes=len(class_names))
model_path = 'resnext101_alllayers.pth'
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

# Transforms (same as training/val transforms)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and Dataloader for TEST set
test_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/test'

class TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, img_name

# Create the dataset and loader
test_dataset = TestDataset(test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          shuffle=False)

# Class names (same order as training classes)
class_names = image_datasets['train'].classes

# Run Inference on the Test Set
predictions = []

with torch.no_grad():
    for inputs, img_names in tqdm(test_loader, desc="Testing"):
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        predicted_label = preds.item()
        predicted_class = class_names[predicted_label]

        # Save predictions
        predictions.append((os.path.basename(img_names[0]).split('.')[0],
                            predicted_class))

predictions.sort(key=lambda x: x[0])

# Save predictions to a CSV
with open('prediction_resnext101_alllayers.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(predictions)

print("Predictions for test set saved to prediction_resnext101_alllayers.csv")


Testing: 100%|██████████| 2344/2344 [03:53<00:00, 10.05it/s]

Predictions for test set saved to prediction_resnext101_alllayers.csv


In [8]:
# Load the Model
model_path = 'resnext101_alllayers.pth'
model.load_state_dict(torch.load(model_path))

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())

# Number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters()
                       if p.requires_grad)

print(f'Total parameters: {total_params}')
print(f'Trainable parameters: {trainable_params}')

Total parameters: 86947236
Trainable parameters: 86947236
